In [3]:
# 路径参数定义
import os
import pandas as pd
import csv
import sys 
import pickle
import numpy as np
from sklearn.model_selection import train_test_split

def ensure_dir_exists(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)
    return dir

data_dir = './origin_data'
work_dir = './work'
tagged_dataset_path = os.path.join(work_dir,'tagged_dataset.csv')
ood_dataset_path = os.path.join(work_dir,'ood_dataset.csv')

model_dir = ensure_dir_exists(os.path.join(work_dir, 'Model'))
knn_model_path = os.path.join(model_dir, 'KNN.pickel')
ood_detector_path = os.path.join(model_dir, 'Maha.pickel')

dataset_dir = ensure_dir_exists(os.path.join(work_dir, 'Dataset'))
test_data_path = os.path.join(dataset_dir, 'test_data.csv')
test_label_path = os.path.join(dataset_dir, 'test_label.csv')
train_data_path = os.path.join(dataset_dir, 'train_data.csv')
train_label_path = os.path.join(dataset_dir, 'train_label.csv')

ood_test_data_path = os.path.join(dataset_dir, 'ood_test_data.csv')
ood_test_label_path = os.path.join(dataset_dir, 'ood_test_label.csv')
ood_train_data_path = os.path.join(dataset_dir, 'ood_train_data.csv')
ood_train_label_path = os.path.join(dataset_dir, 'ood_train_label.csv')

clean_labeled_path = os.path.join(dataset_dir, 'clean_labeled.csv')
clean_labeled_ood_path = os.path.join(dataset_dir, 'clean_labeled_ood.csv')

In [7]:
# 简介：为数据集打tag
# 详请：从数据集目录下读取所有.log文件，根据文件名在csv中加入label，保存至工作区目录(区分OOD与In-distribtuion数据)


files = os.listdir(data_dir)
tagged_data = []
OOD_data = []
tag_file = None
for log_file in files:
    if not log_file.endswith('.log'): # 跳过csv字段文件
        if log_file.endswith('.tag'):
            tag_file = log_file
        continue
    label = log_file[0:log_file.rfind('_')] # 读取文件名
    csv_rows = csv.reader(open(os.path.join(data_dir,log_file),'r'))
    for csv_row in csv_rows:
        csv_row.append(label)
        if label == 'OOD': 
            OOD_data.append(csv_row)
        else:
            tagged_data.append(csv_row)
            
with open(tagged_dataset_path,  'w+') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerows(tagged_data)

with open(ood_dataset_path,  'w+') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerows(OOD_data)

In [8]:
# 简介：读取tagged数据
# 详情：读取tag文件里的csv字段，添上status后读取csv数据

tag_file = os.path.join(data_dir, tag_file)

col_names = next(csv.reader(open(tag_file, 'r')))# 读取tag
col_names.append('label')

data = pd.read_csv(tagged_dataset_path, header=None, names=col_names)
data.info()
ood_data = pd.read_csv(ood_dataset_path, header=None, names=col_names)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543 entries, 0 to 542
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   timestamp             543 non-null    int64  
 1   server_ip             543 non-null    object 
 2   server_port           543 non-null    int64  
 3   client_ip             543 non-null    object 
 4   client_port           543 non-null    int64  
 5   thread_ID             543 non-null    int64  
 6   interval_sec          543 non-null    object 
 7   transfer_bits         543 non-null    int64  
 8   bandwidth_bits        543 non-null    int64  
 9   jitter_ms             543 non-null    float64
 10  cnt_missingdatagrams  543 non-null    int64  
 11  cnt_datagrams         543 non-null    int64  
 12  cnt_errorrate         543 non-null    float64
 13  cnt_outoforders       543 non-null    int64  
 14  label                 543 non-null    object 
dtypes: float64(2), int64(9)

In [9]:
# 简介：数据预处理
# 详情：删除部分无关数据项，对数据集进行划分并保存

def delete_column(data, cols):
    for col in cols:
        data = data.drop(col, axis=1)
    return data

def save_if_not_exists(data, file_path):
    exists = os.path.exists(file_path)
    if not exists:
        data.to_csv(file_path,index=False)
    return exists

label = pd.DataFrame(data['label'])
data = delete_column(data, ['timestamp', 'server_ip', 'server_port', 'client_ip', 'client_port', 'thread_ID', 'label', 'interval_sec'])# 删除无关数据列
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2)

label = pd.DataFrame(ood_data['label'])
ood_data = delete_column(ood_data,['timestamp', 'server_ip', 'server_port', 'client_ip', 'client_port', 'thread_ID', 'label', 'interval_sec'])
ood_X_train, ood_X_test, ood_y_train, ood_y_test = train_test_split(ood_data, label, test_size=0.2)

# 保存数测试集、训练集
# 避免Notebook运行全部cell时重复保存
save_if_not_exists(data, clean_labeled_path)
save_if_not_exists(ood_data, clean_labeled_ood_path)
save_if_not_exists(X_train, train_data_path)
save_if_not_exists(y_train, train_label_path)
save_if_not_exists(X_test, test_data_path)
save_if_not_exists(y_test, test_label_path)

save_if_not_exists(ood_X_train, ood_train_data_path)
save_if_not_exists(ood_y_train, ood_train_label_path)
save_if_not_exists(ood_X_test, ood_test_data_path)
save_if_not_exists(ood_y_test, ood_test_label_path)

True

In [10]:
# 简介：KNN模型训练
# 详情：读取训练集，训练，保存模型
import pickle
import pandas as pd
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

x = pd.read_csv(train_data_path)
y = pd.read_csv(train_label_path)
y = y.values.ravel()

model = KNeighborsClassifier(n_neighbors=6)
model.fit(x, y)

with open(knn_model_path, 'wb') as f:
    pickle.dump(model, f)

In [11]:
# 简介：KNN模型测试
# 详情：读取测试集，测试模型，输出性能报告
model = None
with open(knn_model_path, 'rb') as f:
    model = pickle.load(f)
X_test = pd.read_csv(test_data_path)
y_test = pd.read_csv(test_label_path)
y_pred = model.predict(X_test)


print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None, 
                                              sample_weight=None))

print("Classification Report =\n", metrics.classification_report(y_test, y_pred, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                 digits=2, 
                                                                 output_dict=False))

Confusion Matrix =
 [[ 9  0  1  0  0  0]
 [ 0 66  0  0  0  0]
 [ 0  0  8  0  0  0]
 [ 0  0  0  9  0  0]
 [ 0  0  3  0  2  0]
 [ 0  0  0  0  0 11]]
Classification Report =
                    precision    recall  f1-score   support

         BW_ERROR       1.00      0.90      0.95        10
CONCURRENCY_ERROR       1.00      1.00      1.00        66
       DISK_ERROR       0.67      1.00      0.80         8
    NETWORK_ERROR       1.00      1.00      1.00         9
           NORMAL       1.00      0.40      0.57         5
     SERVER_ERROR       1.00      1.00      1.00        11

         accuracy                           0.96       109
        macro avg       0.94      0.88      0.89       109
     weighted avg       0.98      0.96      0.96       109



In [12]:
# 简介：Mahalanobis OOD探测器
# 详情：训练模型并测试，保存模型
from sklearn import metrics
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegressionCV
import matplotlib.pyplot as plt
import numpy as np
class MahaDetector:
    import pandas as pd
    import numpy as np
    def __init__(self):
        self.threshold = None

    def __data_partition(self, train_data_path, train_label_path):
        X_train = pd.read_csv(train_data_path).values
        y_train = pd.read_csv(train_label_path).values
        label_dict = dict()
        X = list()
        for i in range(0,6):
            X.append(np.zeros((0,7)))

        for i in range(0, X_train.shape[0]):
            _label = str(y_train[i][0])
            if _label not in label_dict:
                label_dict[_label] = len(label_dict)
            element = X_train[i]
            element = element.reshape(1,7)
            X[label_dict[_label]] = np.append(X[label_dict[_label]], element, axis=0)
        return X

    def __get_mahalanobis_dis(self, x, vec):
        xT = x.T  # 求转置
        D = np.cov(xT)  # 求协方差矩阵
        invD = np.linalg.inv(D)  # 协方差逆矩阵
        x_A = vec
        x_B = x.mean(axis=0)
        tp = x_A - x_B
        return np.sqrt(np.dot(np.dot(tp, invD), tp.T))[0]

    def __get_min_maha_dis(self, X, vec):
        vec = vec.reshape(1,7)
        min_dis = None
        for i in range(0,len(X)):
            try:
                if min_dis is None:
                    min_dis = self.__get_mahalanobis_dis(X[0], vec)
                else:
                    tmp = min(self.__get_mahalanobis_dis(X[0], vec), min_dis)
                    min_dis = min_dis if tmp is None else min(min_dis,tmp)
            except:
                pass
        return float(min_dis)
    
    def __get_threshold(self, limit, false_list, true_list):
        step = 0.01
        threshold = step
        cur_acc = 0
        false_list_size = len(false_list)
        true_list_size = len(true_list)
        res = 0
        while threshold < limit:
            try:
                if get_tpr(threshold, false_list, true_list) > 0.95 and get_acc(threshold, false_list, true_list) >= cur_acc:
                    res = threshold
            except:
                pass
            threshold += step
        return res

    def train(self, train_data_path, train_label_path, ood_train_data_path):
        self.partitioned_data = self.__data_partition(train_data_path, train_label_path)
        ood_X_train = pd.read_csv(ood_train_data_path).values
        X_train = pd.read_csv(train_data_path).values
        ood_maha_dis = list()
        maha_dis = list()
        for i in range(0, ood_X_train.shape[0]):
            ood_maha_dis.append(self.__get_min_maha_dis(self.partitioned_data, ood_X_train[i]))
        for i in range(0, X_train.shape[0]):
            maha_dis.append(self.__get_min_maha_dis(self.partitioned_data, X_train[i]))

        self.threshold = self.__get_threshold(min(ood_maha_dis)*1.2, ood_maha_dis, maha_dis)

    def predict(self, X):
        '''
        if X is OOD data return True
        '''
        res = []
        for x in X:
            maha_dis = self.__get_min_maha_dis(self.partitioned_data, x)
            is_ood = (maha_dis >= self.threshold)
            res.append(is_ood)
        return res

def get_tnr(threshold, false_list, true_list):
    true_positive = 0
    true_negative = 0
    false_negative = 0
    false_positive = 0
    
    for false_element in false_list:

        if false_element >= threshold:
            true_negative += 1
        else:
            false_negative += 1

    for true_element in true_list:
        if true_element < threshold:
            true_positive += 1
        else:
            false_positive += 1
    return (true_negative) / (true_negative + false_positive)

def get_tpr(threshold, false_list, true_list):
    true_positive = 0
    true_negative = 0
    false_negative = 0
    false_positive = 0
    
    for false_element in false_list:

        if false_element >= threshold:
            true_negative += 1
        else:
            false_negative += 1

    for true_element in true_list:
        if true_element < threshold:
            true_positive += 1
        else:
            false_positive += 1
    return (true_positive) / (true_positive + false_negative)

def get_acc(threshold, false_list, true_list):
    true_positive = 0
    true_negative = 0
    false_negative = 0
    false_positive = 0
    
    for false_element in false_list:
        if false_element >= threshold:
            true_negative += 1
        else:
            false_negative += 1

    for true_element in true_list:
        if true_element < threshold:
            true_positive += 1
        else:
            false_positive += 1
    return (true_positive+true_negative) / (true_positive + false_negative + true_negative + false_positive)

def get_auc(labels,probs):
    ###initialize
    P = 0
    N = 0
    for i in labels:
        if (i == 1):
            P += 1
        else:
            N += 1
    TP = 0
    FP = 0
    TPR_last = 0
    FPR_last = 0
    AUC = 0
    pair = zip(probs, labels)
    pair = sorted(pair, key=lambda x:x[0], reverse=True)
    i = 0
    while i < len(pair):
        if (pair[i][1] == 1):
            TP += 1
        else:
            FP += 1
        ### maybe have the same probs
        while (i + 1 < len(pair) and pair[i][0] == pair[i+1][0]):
            i += 1
            if (pair[i][1] == 1):
                TP += 1
            else:
                FP += 1
        TPR = TP / P
        FPR = FP / N
        AUC += 0.5 * (TPR + TPR_last) * (FPR - FPR_last)
        TPR_last = TPR
        FPR_last = FPR
        i += 1
    return AUC



MahaD = MahaDetector()
MahaD.train(train_data_path, train_label_path, ood_train_data_path)

# test detector
ood_X_test = pd.read_csv(ood_test_data_path).values
X_test = pd.read_csv(test_data_path).values

labels = []
preds = []
for i in range(0, ood_X_test.shape[0]):
    labels.append(1)
preds = preds + (MahaD.predict(ood_X_test))
for i in range(0, X_test.shape[0]):
    labels.append(0)
preds = preds + (MahaD.predict(X_test))

# metrics

print("Confusion Matrix =\n", metrics.confusion_matrix(labels, preds))

print("Classification Report =\n", metrics.classification_report(labels, preds, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                 digits=2, 
                                                                 output_dict=False))


with open(ood_detector_path,'wb') as f:
    #f.write( pickle.dumps(info) )
    pickle.dump(MahaD,f) 

Confusion Matrix =
 [[88 21]
 [ 1 59]]
Classification Report =
               precision    recall  f1-score   support

           0       0.99      0.81      0.89       109
           1       0.74      0.98      0.84        60

    accuracy                           0.87       169
   macro avg       0.86      0.90      0.87       169
weighted avg       0.90      0.87      0.87       169



请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 